# Data Exploration in Python
### Your name:

Now that you have some basic understanding of how to code in Python, we will now make use of a powerful package in Python called "pandas" to assist us in working with data!

Imagine you are an analyst for Bluebike. Given the growing popularity of Bluebikes as a means for commute and leisure, the city is considering building new stations and/or improving services that move bikes from popular "end stations" to help fill docks at popular "start stations". 

To help them decide where they should put a new dock or where they should improve services, the city wants you to **help them better understand the current state of Bluebike utilization**. Take a moment to think about what questions you might want the answers to in order to chip away at this request. 

YX: If I need to make sure that the right bike docks are filled at the right time, I want some understanding of how the bikes have previously been used to inform my decision of what I think the city should do. 

This is a great example of when data can be helpful -- if I have past information about Bluebike rides, I can try to identify and summarize patterns and trends in bike utilization. 

I may want to think about: 
* When are the bikes being used -- time-wise? date-wise? 
* Where are they being used -- which stations are being docked at, from which stations are bikes being taken from? How frequently? 
* How long is a typical bike ride? Is the duration highly variable (i.e. wide range of times)? 
* Are there other *patterns* underlying the questions I asked above, i.e. other drivers of bike utilization, like weather conditions, holidays, acute events? 

As always, we start by loading in libraries that we plan to use in our coding session.

In [128]:
## Import helpful libraries for EDA
import pandas as pd
import numpy as np

Now, let's load in our data into the data frame. Both of our datasets are comma-separated (i.e. .csv files). Thanks to pandas, this is pretty easy! 

In [ ]:
data = pd.read_csv("/Users/yungexiao/Desktop/Python Workshop 2026/btc_pic_workshop_2026/data/202511-ride-data-day1.csv").iloc[:, 1:] # load in our dataset as a data frame object, drop first column (auto-id column) 

# pd.read_excel is also commonly used when your dataset of interest is and .xlsx file


Now let's get to know our data. 
* Quick look at the first couple of observations. 
* Summary of columns 
* Some important things we might want to know are how many rows and columns (i.e. observations and variables) it contains, and more importantly, what they represent. Remember that this dataset is at the ride-level. 

In [151]:
# quick first look 
data[0:4] # first 5 observations

# how many rows? columns? 
n_rows = data.shape[0] # numbers of rows
n_columns = data.shape[1] # number of columns

column_names = data.columns # names of data columns (variables)
# print(data.dtypes) # column data types 
# print( round(data.describe() , 2) ) # basic descriptive stats summary of each integer or float column (variable), rounded to 2 

# "f" in print allows us to reference objects in our string statement
print(f"Our dataset has {n_rows} rows, representing a unique bluebike rides in November 2025. There are {n_columns} columns, detailing some information about each ride. They are titled: {column_names}.") # print a statement 

Our dataset has 322998 rows, representing a unique bluebike rides in November 2025. There are 19 columns, detailing some information about each ride. They are titled: Index(['ride_id', 'rideable_type', 'member_casual', 'start_station_name',
       'end_station_name', 'start_municipality', 'end_municipality',
       'start_tot_docks', 'end_tot_docks', 'start_dt', 'end_dt', 'start_date',
       'end_date', 'start_day', 'end_day', 'start_hour', 'end_hour',
       'start_dow', 'end_dow'],
      dtype='object').


In [ ]:
# bonus: confirm that the number of rides corresponds to the number of unique ride_id

print( data.ride_id.nunique() == n_rows ) # will return a logical response
print( data['ride_id'].nunique() == n_rows) # another way to extract column from dataset -- index notation

True
True


What if I wanted to know how many unique bluebike stations are in our dataset? 

This task highlights the importance of context, i.e. knowing how a ride is defined; a "ride" is defined by when and where a biker starts and stops their ride, so we need to consider both the start AND stop stations in our calculation.  

In [120]:
# number of unique bluebike stations in our dataset
n_stations = pd.concat([
    data.start_station_name,
    data.end_station_name
]).nunique() # the concatenate function (concat) allows us to "paste" vectors together into a single vector

print(f"There are {n_stations} unique stations in our dataset.")

There are 478 unique stations in our dataset.


Great! Hopefully we have a better idea of what we are working with now. Notice that we have some variables related to date and time that will be a lot easier to work with as datetime variable types. Let's practice converting variable types! 

In [141]:
# convert existing character date-time variable to date-time format, and create meaningful date-time variables
data.start_dt = pd.to_datetime(data.start_dt, format = 'ISO8601').dt.tz_localize('UTC').dt.tz_convert('America/New_York')
data.end_dt = pd.to_datetime(data.end_dt, format = 'ISO8601').dt.tz_localize('UTC').dt.tz_convert('America/New_York')

Nice! You'll see this is helpful because we can now create our first new variable that describes the ride duration. We can do this for minutes AND seconds. 

In [154]:
data['ride_dur'] = (data.end_dt - data.start_dt)
print(data.ride_dur[0]) # default to days hr/min/sec format 

data['ride_dur_sec'] = data.ride_dur.dt.total_seconds() # seconds, keep as exact
data['ride_dur_min'] = round ( data.ride_dur_sec / 60 , 2 ) # minutes, rounded 2 decimal places

print(data.ride_dur_sec[0]) # default to days hr/min/sec format 
print(data.ride_dur_min[0]) # default to days hr/min/sec format 

0 days 00:26:59.849000
1619.849
27.0


We might also want some additional date and time variables so that we can look at bike utilization patterns at different levels, i.e. hour, day, day of week (DOW)

In [161]:
data['start_date'] = pd.to_datetime(data.start_dt.dt.date) # start date of the ride
data['end_date'] =  pd.to_datetime(data.end_dt.dt.date) # end date of the ride 
data['start_day'] = data.start_dt.dt.day # start day (#) of the ride
data['end_day'] = data.end_dt.dt.day # end day (#) of the ride

data['start_hour'] = data.start_dt.dt.hour # start hour of the ride (military time)
data['end_hour'] = data.end_dt.dt.hour # end hour of the ride (military time)

data['start_dow'] = data.start_dt.dt.day_name() # start day of the week of the ride (name) -- day_name is a function so needs the parentheses after
data['end_dow'] = data.end_dt.dt.day_name() # end day of the week of the ride (name)

Cool -- now we know how long each ride was, and extracted helpful elements like the date and hour of the day and DOW in which the ride started and ended. Let's take a look at our current dataset

In [162]:
data[0:4] # first 5 observations

,ride_id,rideable_type,member_casual,start_station_name,end_station_name,start_municipality,end_municipality,start_tot_docks,end_tot_docks,start_dt,...,end_date,start_day,end_day,start_hour,end_hour,start_dow,end_dow,ride_dur,ride_dur_sec,ride_dur_min
0,6D044C1BEC74632F,classic_bike,member,Massachusetts Ave at Boylston St.,Mass Ave T Stop,Boston,Boston,24,16,2025-10-31 20:00:01.822000-04:00,...,2025-10-31,31,31,20,20,Friday,Friday,0 days 00:26:59.849000,1619.849,27.00
1,D577667C39F4AC97,electric_bike,member,Central Sq Post Office / Cambridge City Hall a...,Tremont St at W. Dedham St,Cambridge,Boston,17,15,2025-10-31 20:00:13.446000-04:00,...,2025-10-31,31,31,20,20,Friday,Friday,0 days 00:13:25.984000,805.984,13.43
2,3003D4EFBA42976B,classic_bike,member,Cross St at Hanover St,Lewis Wharf at Atlantic Ave,Boston,Boston,19,31,2025-10-31 20:00:44.502000-04:00,...,2025-10-31,31,31,20,20,Friday,Friday,0 days 00:03:52.678000,232.678,3.88
3,0494595E612E9BB6,classic_bike,casual,Main St at Austin St,Main St at Baldwin St,Boston,Boston,11,19,2025-10-31 20:00:46.747000-04:00,...,2025-10-31,31,31,20,20,Friday,Friday,0 days 00:05:16.601000,316.601,5.28


Woah, glad we checked -- notice how the some rides start/end in October? We are only interested in November for our analysis, so lets practice cleaning data by removing these observations.

In [163]:
data['start_date'].dtype

dtype('<M8[ns]')

In [167]:
# define start and end points of interest 
start = "2025-11-01"
end = "2025-11-30"

data = data.loc[
    data['start_date'].between(start, end) & 
    (data['end_date'] <= end) # include the last day of november, we can use logical statements with datetime variables! 
]

Okay great! I think it's time to get into drawing insights from the data. This next section is dedicated to describing our data (i.e. descriptive statistics). 

Let's recall the purpose of our data exploration by bringing back the guiding questions from earlier!

* When are the bikes being used -- time-wise? date-wise? 

In [ ]:
# Time-wise: Hour of the day 

# Date-wise 1: By date (day-granularity)

# Date-wise 2: By DOW (Mon-Sun)

* Where are they being used -- which stations are being docked at, from which stations are bikes being taken from? How frequently? 

In [ ]:
# By start station 

# By end station 

# Difference in number of start vs end rides, by station? 

# By start municipality 

# By end municipality

# Difference in number of start vs end rides, by municipality? 

* How long is a typical bike ride? Is the duration highly variable (i.e. wide range of times)? 

In [ ]:
# 5-number summary for ride durations (in minutes) 

# Histogram of ride durations 

# Average ride duration, across... 

    ## Date

    ## Hour of the day 

    ## DOW 


* Are there other *patterns* underlying the questions I asked above, i.e. other drivers of bike utilization, like weather conditions, holidays, acute events? 

In [ ]:
# Merge with weather data (maybe save for next class)

# Create holiday indicator 